In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.spatial import distance
import random
from pyxdameraulevenshtein import damerau_levenshtein_distance
import math

pd.set_option('display.max_rows', 500)

In [38]:
objects = ['t', 'n', 's', 'p', 'c']

coordinates = {'c': (1,4,4),
              'n': (1,1,2),
              'p': (1,4,4),
              's': (1,1,2),
              't': (1,1,2),
              'start': (2,1,2),
              'table': (4,1,2)}

c1 = {'c': 1.2,
    'n': 1.0,
    'p': 1.2,
    's': 1.2,
    't': 1.0}

k1 = {'c': 1.0,
    'n': 0.95,
    'p': 0.95,
    's': 1.0,
    't': 0.9}

c0 = {'c': 1.0,
    'n': 1.0,
    'p': 1.0,
    's': 1.0,
    't': 1.0}

k0 = {'c': 1.0,
    'n': 1.0,
    'p': 1.0,
    's': 1.0,
    't': 1.0}

start_coordinates = [[2,1,2],[4,3,2],[4,3,2],[4,3,2],[4,3,2]]

In [39]:
def predict_sequence(possible_items, coordinates, c, k, start_coordinates, dimensions=[3,]):
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    
    if dimensions[0] == 3: # no changes if 3D
        pass
        
    elif dimensions[0] == 2: # 2D: remove obsolete coordinate
        if dimensions[1] == 'xy':
            coords = {key: value[:-1] for key, value in coords.items()}
            start_coords = [x[:-1] for x in start_coords]
                
        elif dimensions[1] == 'xz':
            #coords = {key: value for key, value in coords.items()}
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                coords[key] = new_value

            for x in range (0, len(start_coords)):
                start_coords[x] = [start_coords[x][0], start_coords[x][-1]]
        
        elif dimensions[1] == 'yz':
            coords = {key: value[1:] for key, value in coords.items()}
            start_coords = [x[1:] for x in start_coords]
                
    elif dimensions[0] == 1: # 1D: choose appropriate coordinate
        if dimensions[1] == 'x':
            coords = {key: value[0] for key, value in coords.items()}
            start_coords = [x[0] for x in start_coords]
        
        elif dimensions[1] == 'y':
            coords = {key: value[1] for key, value in coords.items()}
            start_coords = [x[1] for x in start_coords]
        
        elif dimensions[1] == 'z':
            coords = {key: value[2] for key, value in coords.items()}
            start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                start_coords[coord_index], 
                                coords[obj])
                                ) * k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [40]:
predict_sequence(objects,coordinates, c1, k1, start_coordinates, dimensions=[2,'xy'])

['t', 'n', 'p', 'c', 's']

In [45]:
edit_list = []

for x in range(0,1000):
    result = ''.join(predict_sequence(objects,coordinates, c1, k1, start_coordinates, dimensions=[3,'xyz']))
    #print(result)
    dl = damerau_levenshtein_distance('tnspc',result)
    edit_list.append(dl)
    
avg = np.mean(edit_list)
avg

0.0

In [197]:
# create vectors from 3D points, calculate diff vector

vec1 = np.array([3,3,1])
vec2 = np.array([4,1,3])
vec = vec2 - vec1

In [282]:
# generate test plane (x,y)

p1 = np.array([2,1,0])
p2 = np.array([5,2,0])
p3 = np.array([3,1,0])

In [283]:
# calculate vectors in test plane
v1 = p3 - p1
v2 = p2 - p1

In [297]:
# calculate crossproduct (vector normal to plane)
crossprod = np.cross(v1, v2)

# get norm of crossproduct
n_norm = np.sqrt(sum(crossprod**2))

# get projection
projection = (np.dot(vec, crossprod)/n_norm**2)*crossprod

# calculate projection of vec on plane
vec_new = vec - projection

In [290]:
np.linalg.norm(vec_new)

2.23606797749979

In [276]:
distance.euclidean(vec1, vec2)

3.0